In [8]:
import numpy as np
import pandas as pd
from PIL import Image, ImageFilter
import os
import cv2

In [2]:
num_fault = []
num_horizon = []
num_seismic = []
indeces = [v for v in range(1000, 1441)]
for name in os.listdir('raw_fault'):
    if name == '.ipynb_checkpoints':
        continue

    num_fault.append(int(name.split('-')[1].split('.')[0]))

for name in os.listdir('raw_horizon'):
    if name == '.ipynb_checkpoints':
        continue
        
    num_horizon.append(int(name.split('-')[1].split('.')[0]))

for name in os.listdir('raw_seismic'):
    if name == '.ipynb_checkpoints':
        continue
        
    num_seismic.append(int(name.split('-')[1].split('.')[0]))

df = pd.DataFrame({
    'RAW_SEISMIC': [f"seismic-{x}.png" if x in num_seismic else None for x in indeces],
    'RAW_FAULT': [f"fault-{x}.png" if x in num_fault else None for x in indeces],
    'RAW_HORIZON': [f"horizon-{x}.png" if x in num_horizon else None for x in indeces]
})

In [10]:
df

,RAW_SEISMIC,RAW_FAULT,RAW_HORIZON
0,seismic-1000.png,fault-1000.png,horizon-1000.png
1,seismic-1001.png,fault-1001.png,horizon-1001.png
2,seismic-1002.png,fault-1002.png,horizon-1002.png
3,seismic-1003.png,fault-1003.png,horizon-1003.png
4,seismic-1004.png,fault-1004.png,horizon-1004.png
...,...,...,...
436,seismic-1436.png,fault-1436.png,horizon-1436.png
437,seismic-1437.png,fault-1437.png,horizon-1437.png
438,seismic-1438.png,fault-1438.png,horizon-1438.png
439,seismic-1439.png,fault-1439.png,horizon-1439.png


In [11]:
df = df.dropna().reset_index(drop=True)

In [31]:

def sample_window_from_image(image_path, x, y, width, height, flip=False):
    """
    Samples a specific window from a PNG image based on coordinates.

    Parameters:
    - image_path (str): The file path to the PNG image.
    - x (int): The x-coordinate of the top-left corner of the window.
    - y (int): The y-coordinate of the top-left corner of the window.
    - width (int): The width of the window.
    - height (int): The height of the window.

    Returns:
    - Image: A Pillow Image object representing the sampled window.
    """
    # Open the image using Pillow
    with Image.open(image_path) as img:
        # Define the box to crop: (left, upper, right, lower)
        box = (x, y, x + width, y + height)
        
        # Crop the image
        window = img.crop(box)
        if flip:
            window = window.transpose(Image.FLIP_LEFT_RIGHT)
        
    return window


# Input values
output_count = 10000
whole_width = 4166
whole_height = 2664
width = 512
height = 512


np.random.seed(42)
index = np.random.randint(len(df), high=None, size=output_count)
x_vals = np.random.randint(whole_width-width, high=None, size=output_count)
y_vals = np.random.randint(whole_height-height, high=None, size=output_count)
flip_vals = np.random.randint(2, high=None, size=output_count)
for i in range(output_count):
    for fol in ["RAW_SEISMIC","RAW_FAULT","RAW_HORIZON"]:
        infilepath = "{}/{}".format(fol.lower(), df.at[index[i],fol])
        x = x_vals[i]
        y = y_vals[i]
        outfilepath = "aug_{}/{}_{}_{}_{}.png".format(fol.lower(), df.at[index[i],fol].replace(".png",""), x, y, flip_vals[i])
    
    
        # Sample the window from the image
        sampled_window = sample_window_from_image(infilepath, x, y, width, height, flip_vals[i])
    
        # Show the sampled window
        # sampled_window.show()  # This will display the cropped window
        # break
    
    # # Optionally save the sampled window
        sampled_window.save(outfilepath)
    print(f"Sampled window saved to: {outfilepath}")

Sampled window saved to: aug_raw_horizon/horizon-1102_1451_514_0.png
Sampled window saved to: aug_raw_horizon/horizon-1427_1854_1837_1.png
Sampled window saved to: aug_raw_horizon/horizon-1329_3300_575_0.png
Sampled window saved to: aug_raw_horizon/horizon-1106_1789_1165_0.png
Sampled window saved to: aug_raw_horizon/horizon-1071_3036_902_1.png
Sampled window saved to: aug_raw_horizon/horizon-1188_3281_414_1.png
Sampled window saved to: aug_raw_horizon/horizon-1020_3502_1104_1.png
Sampled window saved to: aug_raw_horizon/horizon-1102_3365_762_0.png
Sampled window saved to: aug_raw_horizon/horizon-1121_742_562_0.png
Sampled window saved to: aug_raw_horizon/horizon-1230_1776_1515_1.png
Sampled window saved to: aug_raw_horizon/horizon-1402_3653_1934_1.png
Sampled window saved to: aug_raw_horizon/horizon-1087_3175_1369_1.png
Sampled window saved to: aug_raw_horizon/horizon-1099_203_203_1.png
Sampled window saved to: aug_raw_horizon/horizon-1438_1076_1576_1.png
Sampled window saved to: aug_

KeyboardInterrupt: 

In [14]:
for name in os.listdir('aug_raw_fault'):
    if name == ".ipynb_checkpoints":
        continue
    if name == 'fault-1024_482_757_1.png':
        filename = name.split('.')[0]
        img = cv2.imread('aug_raw_fault/' + name)
    
        # Convert to grayscale
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
        # Apply binary threshold to get the black pixels
        _, binary_mask = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY_INV)
    
        # Convert to binary format (0 and 1)
        binary_mask = binary_mask // 255
    
        # Save the binary mask as a .npy file
        np.save('aug_fault_mask/' + filename + '.npy', binary_mask)

In [24]:
for name in os.listdir('aug_raw_horizon'):
    if name == ".ipynb_checkpoints":
        continue
    filename = name.split('.')[0]
    img = cv2.imread('aug_raw_horizon/' + name)

    # Convert to grayscale
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply binary thresholding
    _, binary_image = cv2.threshold(gray_image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    binary_image = cv2.bitwise_not(binary_image)

    # Convert to binary format (0 and 1)
    binary_mask = binary_image // 255

    # Save the binary mask as a .npy file
    np.save('aug_horizon_mask/' + filename + '.npy', binary_mask)

In [21]:

for name in os.listdir('aug_horizon_mask'):
    if name == ".ipynb_checkpoints":
        continue
    if "_" not in name:
        continue
    code = name.replace("horizon-","").replace(".npy","")
    img = Image.open('aug_raw_fault/fault-' + code+".png")
    img = img.filter(ImageFilter.GaussianBlur(radius=4))
    img = img.convert("L")
    img = img.point( lambda p: 255 if p > 250 else 0 )
    img = img.convert('1')
    # Display the Box Blurred image
    img.save('aug_raw_fault_filter/fault-' + code+".png")

In [22]:
for name in os.listdir('aug_raw_fault_filter'):
    if name == ".ipynb_checkpoints":
        continue
    filename = name.split('.')[0]
    img = cv2.imread('aug_raw_fault_filter/' + name)

    # Convert to grayscale
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply binary threshold to get the black pixels
    _, binary_mask = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY_INV)

    # Convert to binary format (0 and 1)
    binary_mask = binary_mask // 255

    # Save the binary mask as a .npy file
    np.save('aug_fault_mask_filter/' + filename + '.npy', binary_mask)

In [15]:
code = '1024_482_757_1'
img = cv2.imread(f'aug_raw_seismic/seismic-{code}.png')
fault_mask = np.load(f'aug_fault_mask/fault-{code}.npy')
inverted_fault_mask = cv2.bitwise_not(fault_mask * 255)

# Convert binary image to a 3-channel image for overlay (BGR format)
fault_mask_bgr = cv2.cvtColor(inverted_fault_mask, cv2.COLOR_GRAY2BGR)

overlay = cv2.addWeighted(img, 0.5, fault_mask_bgr, 0.5, 0)

cv2.imwrite("overlay_img_fault2.png", overlay)

True

In [22]:
code

'1024_482_757_1'

In [18]:
img.shape

(512, 512, 3)

In [11]:
code = '1024_482_757_1'
img = cv2.imread(f'aug_raw_seismic/seismic-{code}.png')
horizon_mask = np.load(f'aug_horizon_mask/horizon-{code}.npy')
inverted_horizon_mask = cv2.bitwise_not(horizon_mask * 255)

# Convert binary image to a 3-channel image for overlay (BGR format)
horizon_mask_bgr = cv2.cvtColor(inverted_horizon_mask, cv2.COLOR_GRAY2BGR)

overlay = cv2.addWeighted(img, 0.5, horizon_mask_bgr, 0.5, 0)

cv2.imwrite("overlay_img_horizon.png", overlay)

True

In [30]:
len(os.listdir('aug_fault_mask_filter'))

7456

# Get only if contain filter

In [50]:
for name in os.listdir('aug_fault_mask_filter'):
    if name == ".ipynb_checkpoints":
        continue
    code = name.replace("fault-","").replace(".npy", "")

    fault_mask = np.load(f'aug_fault_mask_filter/{name}')
    if np.max(fault_mask) >= 1:
        print(code)
        print(os.path.isfile(f'aug_fault_mask_filter/fault-{code}.npy'))
        print(os.path.isfile(f'aug_raw_seismic/seismic-{code}.png'))
        print(os.path.isfile(f'aug_raw_fault_filter/fault-{code}.png'))
        if os.path.isfile(f'aug_fault_mask_filter/fault-{code}.npy') and os.path.isfile(f'aug_raw_seismic/seismic-{code}.png') and os.path.isfile(f'aug_raw_fault_filter/fault-{code}.png'):
            os.popen(f'cp aug_fault_mask_filter/fault-{code}.npy aug_fault_mask_filter_hasfault/fault-{code}.npy') 
            os.popen(f'cp aug_raw_seismic/seismic-{code}.png aug_raw_seismic_hasfault/seismic-{code}.png') 
            os.popen(f'cp aug_raw_fault_filter/fault-{code}.png aug_raw_fault_filter_hasfault/fault-{code}.png') 
            print(code)


1392_3066_582_0
True
True
True
1392_3066_582_0
1312_1045_814_0
True
True
True
1312_1045_814_0
1024_319_614_0
True
True
True
1024_319_614_0
1264_298_434_1
True
True
True
1264_298_434_1
1288_1075_380_1
True
True
True
1288_1075_380_1
1400_624_713_0
True
True
True
1400_624_713_0
1416_791_463_0
True
True
True
1416_791_463_0
1240_1240_398_0
True
True
True
1240_1240_398_0
1256_1052_539_0
True
True
True
1256_1052_539_0
1059_697_632_0
True
True
True
1059_697_632_0
1216_1041_392_1
True
True
True
1216_1041_392_1
1167_747_623_1
True
True
True
1167_747_623_1
1417_2612_333_1
True
True
True
1417_2612_333_1
1324_1160_25_0
True
True
True
1324_1160_25_0
1296_1127_190_0
True
True
True
1296_1127_190_0
1312_2350_1155_1
True
True
True
1312_2350_1155_1
1248_567_45_1
True
True
True
1248_567_45_1
1240_3041_579_0
True
True
True
1240_3041_579_0
1252_1543_159_0
True
True
True
1252_1543_159_0
1434_2761_175_0
True
True
True
1434_2761_175_0
1113_2801_536_0
True
True
True
1113_2801_536_0
1240_278_785_0
True
True
True

In [52]:
len(os.listdir('aug_fault_mask_filter_hasfault'))

770

In [53]:
len(os.listdir('aug_raw_fault_filter_hasfault'))

770

In [55]:
len(os.listdir('aug_raw_seismic_hasfault'))

770

In [ ]:

for fol in ["RAW_SEISMIC","RAW_FAULT","RAW_HORIZON"]:
    infilepath = "{}/{}".format(fol.lower(), df.at[index[i],fol])
    x = x_vals[i]
    y = y_vals[i]
    outfilepath = "aug_{}/{}_{}_{}_{}.png".format(fol.lower(), df.at[index[i],fol].replace(".png",""), x, y, flip_vals[i])


    # Sample the window from the image
    sampled_window = sample_window_from_image(infilepath, x, y, width, height, flip_vals[i])

    # Show the sampled window
    # sampled_window.show()  # This will display the cropped window
    # break

# # Optionally save the sampled window
    sampled_window.save(outfilepath)
print(f"Sampled window saved to: {outfilepath}")